In [1]:
import pandas as pd

### pandas로 dataframe형태 데이터 가져오기

In [2]:
train = pd.read_csv('titanic/train.csv')
test = pd.read_csv('titanic/test.csv')

### 나이와 요금 그룹핑 시켜서 돌려보기

In [3]:
def grouping(csvdata):
    csvdata['Age'].fillna(0, inplace = True)
    agelist = []
    for row in csvdata['Age']:
        row = int(row/10)
        agelist.append(row)

    csvdata['NewAge'] = agelist
    
    csvdata['Fare'].fillna(0, inplace = True)
    farelist = []
    for row in csvdata['Fare']:
        row = int(row/10)
        farelist.append(row)

    csvdata['NewFare'] = farelist


In [4]:
grouping(train)
grouping(test)

### 비율 그래프를 고려해 본 결과, 성별과 나이(나이대로 변경)에서 뚜렷한 상관관계가 발견됐으며 pclass, embarked도 상대적으로 작지만 관계가 있는것으로 분석된다.
### 성별의 경우 남성의 경우가 인원대비로 보았을때도 상당 수가 사망했으며, 나이의 경우 생존자에는 80대가 없지만 사망자에는 70대가 없는것으로 나왔다.

## 첫번째, 접근 방식으로는 앞서 정한 4가지의 조건을 활용하여 다중 분류를 진행해보려한다~

In [5]:
def preparing(data):
    data.drop(['PassengerId','Name','Age','SibSp','Parch','Ticket','Fare','Cabin','NewFare'], axis = 1, inplace = True)
    data['Sex'] = data['Sex'].apply(lambda x: len(x))
    templist = []
    
    for row in data['Embarked']:
        if(row == 'S'):
            templist.append(0)
        elif(row == 'Q'):
            templist.append(1)
        else:
            templist.append(2)
    data['NewEmb'] = templist
    data.drop('Embarked', axis=1, inplace = True)    

In [6]:
preparing(train)
preparing(test)
train

,Survived,Pclass,Sex,NewAge,NewEmb
0,0,3,4,2,0
1,1,1,6,3,2
2,1,3,6,2,0
3,1,1,6,3,0
4,0,3,4,3,0
5,0,3,4,0,1
6,0,1,4,5,0
7,0,3,4,0,0
8,1,3,6,2,0
9,1,2,6,1,2


In [7]:
def divideData(data):
    x_data = data.iloc[:, 1:5]
    y_data = data.iloc[:,0:1]
    x_data = x_data.to_numpy()
    y_data = y_data.to_numpy()
    idx = int(len(x_data)*0.8)
    xtr = x_data[:idx]
    ytr = y_data[:idx]
    xte = x_data[idx:]
    yte = y_data[idx:]
    return xtr, ytr, xte, yte

In [8]:
train_x, train_y, test_x, test_y = divideData(train)


In [9]:
len(test_x)

179

In [10]:
from keras.models import Sequential
from keras.layers import Dense, Activation

model = Sequential()
model.add(Dense(10, activation = 'relu', input_shape = (4,)))
model.add(Dense(5, activation = 'relu', input_shape = (10,)))
model.add(Dense(1, activation = 'sigmoid', input_shape = (5,)))

model.compile(optimizer='adam', loss = 'mse', metrics = ['accuracy'])

Using TensorFlow backend.


In [11]:
model.fit(train_x, train_y, epochs = 1000, batch_size= 89)

Epoch 1/1000
712/712 [==============================] - 0s 203us/step - loss: 0.4348 - accuracy: 0.3904
Epoch 2/1000
712/712 [==============================] - 0s 21us/step - loss: 0.4053 - accuracy: 0.3904
Epoch 3/1000
712/712 [==============================] - 0s 22us/step - loss: 0.3755 - accuracy: 0.3904
Epoch 4/1000
712/712 [==============================] - 0s 24us/step - loss: 0.3469 - accuracy: 0.3904
Epoch 5/1000
712/712 [==============================] - 0s 22us/step - loss: 0.3211 - accuracy: 0.3834
Epoch 6/1000
712/712 [==============================] - 0s 22us/step - loss: 0.3003 - accuracy: 0.3778
Epoch 7/1000
712/712 [==============================] - 0s 22us/step - loss: 0.2843 - accuracy: 0.3624
Epoch 8/1000
712/712 [==============================] - 0s 24us/step - loss: 0.2715 - accuracy: 0.3610
Epoch 9/1000
712/712 [==============================] - 0s 21us/step - loss: 0.2621 - accuracy: 0.3694
Epoch 10/1000
712/712 [==============================] - 0s 20us/step - 

712/712 [==============================] - 0s 15us/step - loss: 0.1590 - accuracy: 0.7851
Epoch 80/1000
712/712 [==============================] - 0s 13us/step - loss: 0.1586 - accuracy: 0.7837
Epoch 81/1000
712/712 [==============================] - 0s 17us/step - loss: 0.1584 - accuracy: 0.7851
Epoch 82/1000
712/712 [==============================] - 0s 14us/step - loss: 0.1581 - accuracy: 0.7879
Epoch 83/1000
712/712 [==============================] - 0s 11us/step - loss: 0.1579 - accuracy: 0.7879
Epoch 84/1000
712/712 [==============================] - 0s 14us/step - loss: 0.1573 - accuracy: 0.7879
Epoch 85/1000
712/712 [==============================] - 0s 14us/step - loss: 0.1572 - accuracy: 0.7865
Epoch 86/1000
712/712 [==============================] - 0s 14us/step - loss: 0.1568 - accuracy: 0.7865
Epoch 87/1000
712/712 [==============================] - 0s 14us/step - loss: 0.1566 - accuracy: 0.7851
Epoch 88/1000
712/712 [==============================] - 0s 14us/step - loss: 

712/712 [==============================] - 0s 17us/step - loss: 0.1471 - accuracy: 0.7865
Epoch 158/1000
712/712 [==============================] - 0s 18us/step - loss: 0.1473 - accuracy: 0.7865
Epoch 159/1000
712/712 [==============================] - 0s 15us/step - loss: 0.1471 - accuracy: 0.7865
Epoch 160/1000
712/712 [==============================] - 0s 15us/step - loss: 0.1469 - accuracy: 0.7865
Epoch 161/1000
712/712 [==============================] - 0s 17us/step - loss: 0.1469 - accuracy: 0.7865
Epoch 162/1000
712/712 [==============================] - 0s 14us/step - loss: 0.1467 - accuracy: 0.7865
Epoch 163/1000
712/712 [==============================] - 0s 14us/step - loss: 0.1467 - accuracy: 0.7865
Epoch 164/1000
712/712 [==============================] - 0s 15us/step - loss: 0.1467 - accuracy: 0.7865
Epoch 165/1000
712/712 [==============================] - 0s 17us/step - loss: 0.1467 - accuracy: 0.7865
Epoch 166/1000
712/712 [==============================] - 0s 16us/step

712/712 [==============================] - 0s 15us/step - loss: 0.1432 - accuracy: 0.7949
Epoch 235/1000
712/712 [==============================] - 0s 17us/step - loss: 0.1433 - accuracy: 0.7921
Epoch 236/1000
712/712 [==============================] - 0s 17us/step - loss: 0.1431 - accuracy: 0.7921
Epoch 237/1000
712/712 [==============================] - 0s 14us/step - loss: 0.1433 - accuracy: 0.7978
Epoch 238/1000
712/712 [==============================] - 0s 14us/step - loss: 0.1432 - accuracy: 0.7992
Epoch 239/1000
712/712 [==============================] - 0s 17us/step - loss: 0.1430 - accuracy: 0.7949
Epoch 240/1000
712/712 [==============================] - 0s 14us/step - loss: 0.1430 - accuracy: 0.7949
Epoch 241/1000
712/712 [==============================] - 0s 14us/step - loss: 0.1432 - accuracy: 0.7949
Epoch 242/1000
712/712 [==============================] - 0s 15us/step - loss: 0.1429 - accuracy: 0.7949
Epoch 243/1000
712/712 [==============================] - 0s 15us/step

712/712 [==============================] - 0s 15us/step - loss: 0.1415 - accuracy: 0.7949
Epoch 313/1000
712/712 [==============================] - 0s 15us/step - loss: 0.1414 - accuracy: 0.7949
Epoch 314/1000
712/712 [==============================] - 0s 17us/step - loss: 0.1415 - accuracy: 0.7949
Epoch 315/1000
712/712 [==============================] - 0s 15us/step - loss: 0.1417 - accuracy: 0.7949
Epoch 316/1000
712/712 [==============================] - 0s 15us/step - loss: 0.1414 - accuracy: 0.7949
Epoch 317/1000
712/712 [==============================] - 0s 14us/step - loss: 0.1416 - accuracy: 0.7935
Epoch 318/1000
712/712 [==============================] - ETA: 0s - loss: 0.1721 - accuracy: 0.76 - 0s 14us/step - loss: 0.1414 - accuracy: 0.7949
Epoch 319/1000
712/712 [==============================] - 0s 17us/step - loss: 0.1416 - accuracy: 0.7949
Epoch 320/1000
712/712 [==============================] - 0s 15us/step - loss: 0.1414 - accuracy: 0.7949
Epoch 321/1000
712/712 [====

...

712/712 [==============================] - 0s 14us/step - loss: 0.1392 - accuracy: 0.7963
Epoch 623/1000
712/712 [==============================] - 0s 13us/step - loss: 0.1392 - accuracy: 0.8146
Epoch 624/1000
712/712 [==============================] - 0s 14us/step - loss: 0.1393 - accuracy: 0.8062
Epoch 625/1000
712/712 [==============================] - 0s 14us/step - loss: 0.1392 - accuracy: 0.8034
Epoch 626/1000
712/712 [==============================] - 0s 11us/step - loss: 0.1392 - accuracy: 0.8090
Epoch 627/1000
712/712 [==============================] - 0s 14us/step - loss: 0.1392 - accuracy: 0.8020
Epoch 628/1000
712/712 [==============================] - 0s 15us/step - loss: 0.1393 - accuracy: 0.8020
Epoch 629/1000
712/712 [==============================] - 0s 18us/step - loss: 0.1393 - accuracy: 0.8062
Epoch 630/1000
712/712 [==============================] - 0s 17us/step - loss: 0.1392 - accuracy: 0.8062
Epoch 631/1000
712/712 [==============================] - 0s 14us/step

712/712 [==============================] - 0s 14us/step - loss: 0.1390 - accuracy: 0.8090
Epoch 701/1000
712/712 [==============================] - 0s 13us/step - loss: 0.1390 - accuracy: 0.8104
Epoch 702/1000
712/712 [==============================] - 0s 18us/step - loss: 0.1389 - accuracy: 0.8034
Epoch 703/1000
712/712 [==============================] - 0s 17us/step - loss: 0.1389 - accuracy: 0.8160
Epoch 704/1000
712/712 [==============================] - 0s 15us/step - loss: 0.1390 - accuracy: 0.8146
Epoch 705/1000
712/712 [==============================] - 0s 15us/step - loss: 0.1391 - accuracy: 0.8062
Epoch 706/1000
712/712 [==============================] - 0s 15us/step - loss: 0.1394 - accuracy: 0.8034
Epoch 707/1000
712/712 [==============================] - 0s 14us/step - loss: 0.1389 - accuracy: 0.8104
Epoch 708/1000
712/712 [==============================] - 0s 15us/step - loss: 0.1391 - accuracy: 0.7992
Epoch 709/1000
712/712 [==============================] - 0s 15us/step

712/712 [==============================] - 0s 17us/step - loss: 0.1386 - accuracy: 0.8146
Epoch 779/1000
712/712 [==============================] - 0s 15us/step - loss: 0.1386 - accuracy: 0.8090
Epoch 780/1000
712/712 [==============================] - 0s 15us/step - loss: 0.1386 - accuracy: 0.8034
Epoch 781/1000
712/712 [==============================] - 0s 14us/step - loss: 0.1386 - accuracy: 0.7949
Epoch 782/1000
712/712 [==============================] - 0s 13us/step - loss: 0.1388 - accuracy: 0.8132
Epoch 783/1000
712/712 [==============================] - 0s 15us/step - loss: 0.1386 - accuracy: 0.8118
Epoch 784/1000
712/712 [==============================] - 0s 14us/step - loss: 0.1386 - accuracy: 0.8006
Epoch 785/1000
712/712 [==============================] - 0s 14us/step - loss: 0.1386 - accuracy: 0.8048
Epoch 786/1000
712/712 [==============================] - 0s 14us/step - loss: 0.1385 - accuracy: 0.8076
Epoch 787/1000
712/712 [==============================] - 0s 17us/step

712/712 [==============================] - 0s 13us/step - loss: 0.1383 - accuracy: 0.8146
Epoch 856/1000
712/712 [==============================] - 0s 17us/step - loss: 0.1383 - accuracy: 0.8132
Epoch 857/1000
712/712 [==============================] - 0s 15us/step - loss: 0.1382 - accuracy: 0.8160
Epoch 858/1000
712/712 [==============================] - 0s 14us/step - loss: 0.1383 - accuracy: 0.8062
Epoch 859/1000
712/712 [==============================] - 0s 13us/step - loss: 0.1384 - accuracy: 0.8076
Epoch 860/1000
712/712 [==============================] - 0s 14us/step - loss: 0.1381 - accuracy: 0.8174
Epoch 861/1000
712/712 [==============================] - 0s 14us/step - loss: 0.1385 - accuracy: 0.8146
Epoch 862/1000
712/712 [==============================] - 0s 15us/step - loss: 0.1381 - accuracy: 0.8146
Epoch 863/1000
712/712 [==============================] - 0s 14us/step - loss: 0.1383 - accuracy: 0.8104
Epoch 864/1000
712/712 [==============================] - 0s 15us/step

712/712 [==============================] - 0s 13us/step - loss: 0.1380 - accuracy: 0.8020
Epoch 934/1000
712/712 [==============================] - 0s 14us/step - loss: 0.1379 - accuracy: 0.8146
Epoch 935/1000
712/712 [==============================] - 0s 15us/step - loss: 0.1379 - accuracy: 0.8146
Epoch 936/1000
712/712 [==============================] - 0s 14us/step - loss: 0.1381 - accuracy: 0.8146
Epoch 937/1000
712/712 [==============================] - 0s 13us/step - loss: 0.1380 - accuracy: 0.8146
Epoch 938/1000
712/712 [==============================] - 0s 17us/step - loss: 0.1380 - accuracy: 0.8118
Epoch 939/1000
712/712 [==============================] - 0s 17us/step - loss: 0.1379 - accuracy: 0.8146
Epoch 940/1000
712/712 [==============================] - 0s 15us/step - loss: 0.1382 - accuracy: 0.8146
Epoch 941/1000
712/712 [==============================] - 0s 15us/step - loss: 0.1379 - accuracy: 0.8118
Epoch 942/1000
712/712 [==============================] - 0s 17us/step

In [12]:
score = model.evaluate(test_x, test_y, batch_size=209)

179/179 [==============================] - 0s 72us/step


In [13]:
score

[0.12683741748332977, 0.826815664768219]

### 제출 결과 0.75119 